In [6]:
from PIL import Image, ImageOps
import numpy as np
from scipy.ndimage import generate_binary_structure, label

"""
    Isolate a specific color in an image and save it as a new image.

    Args:
        image_src (str): The path to the input image.
        image_out (str): The path to save the isolated image.
        target_color (tuple): The RGB color tuple to isolate (e.g., (255, 0, 0) for red).
        tolerance (int, optional): Tolerance level for color matching. Default is 20.
"""

def isolate_map(image_src, image_out, target_color, min_area=100, tolerance=20):
    # Open the PNG image
    input_image = Image.open(image_src)

    # Create a mask to isolate the target color with tolerance
    def create_color_mask(image, target_color, tolerance):
        image = image.convert("RGBA")
        data = image.getdata()

        new_data = []
        for item in data:
            r, g, b, a = item[:4]
            tr, tg, tb = target_color

            # Check if the color is within the tolerance range
            if (
                abs(r - tr) <= tolerance
                and abs(g - tg) <= tolerance
                and abs(b - tb) <= tolerance
            ):
                new_data.append(item)
            else:
                new_data.append((0, 0, 0, 0))  # Make other colors transparent

        image.putdata(new_data)
        return image

    # Create the color mask with tolerance
    color_mask = create_color_mask(input_image, target_color, tolerance)
    
    # Convert the color mask to a binary array (black and white)
    color_mask_array = np.array(color_mask)

    # Apply morphological operations to filter out small regions
    structure = generate_binary_structure(2, 2)
    labeled_array, num_features = label(color_mask_array[:, :, 3], structure)
    for feature in range(1, num_features + 1):
        region = labeled_array == feature
        if region.sum() < min_area:
            color_mask_array[np.where(region)] = (0, 0, 0, 0)  # Set the region to transparent

    # Convert the filtered NumPy array back to an RGBA image
    color_mask_filtered = Image.fromarray(color_mask_array)

    # Add a white background
    width, height = color_mask_filtered.size
    background_color = (255, 255, 255)
    background = Image.new("RGBA", (width, height), background_color)
    background.paste(color_mask_filtered, (0, 0), color_mask_filtered)

    # Save the isolated color with white background
    background.save(image_out)

    # Close the original image and the color mask
    input_image.close()
    color_mask_filtered.close()
    background.close()

In [7]:
isolate_map('simple_floor_1.png', 'iso_floor_1.png', (0, 0, 255))
isolate_map('simple_floor_plan.png', 'iso_floor_2.png', (69, 99, 137))
isolate_map('school_floor_plan.png', 'iso_school_1.png', (0,75,133))